# Create a catalog order

## Installation

[Install](https://sdk.up42.com/installation/) the SDK. The package requires Python ≥ 3.9.

In [1]:
!pip install up42-py
import up42

## Authentication

Use your credentials to [authenticate](https://sdk.up42.com/authentication).

In [2]:
# Create a configuration file and use the following code for it:
# {
#   "username": "<your-email-address>",
#   "password": "<your-password>"
# }

up42.authenticate(cfg_file="credentials.json")

2023-12-06 16:43:14,892 - Got credentials from config file.
2023-12-06 16:43:15,609 - Authentication with UP42 successful!


## Step 1. Choose a catalog collection

Choose a catalog collection and get its `data_product_id` for ordering.

In [3]:
catalog = up42.initialize_catalog()
products = catalog.get_data_products(basic=True)
products

Choose a data product and copy its ID.

In [4]:
# The following value is the product id for Pléiades catalog — Analytic:
data_product_id = "4f1b2f62-98df-4c74-81f4-5dce45deee99"

## Step 2. Request access

If you want to order the chosen collection for the first time, you need to request access to it. For more information on access requests, see [Restrictions](https://docs.up42.com/getting-started/restrictions#tasking-collections).

An email from the Customer Success team usually takes up to 3 days. You can review your access request status on the [Access requests](https://console.up42.com/settings/access) page.

## Step 3. Accept a EULA

If you want to order the chosen collection for the first time, you need to accept its end-user license agreement (EULA). For more information on license agreements, see [EULAs](https://docs.up42.com/getting-started/account/eulas).

## Step 4. Search the catalog

Specify search parameters to find a full scene that fits your requirements.

In [5]:
geometry = up42.get_example_aoi()

search_parameters = catalog.construct_search_parameters(
    collections=["phr"],
    geometry=geometry,
    start_date="2022-06-01",
    end_date="2022-12-31",
    max_cloudcover=20,
    limit=10,
)

search_results_df = catalog.search(search_parameters)

2023-12-06 16:54:44,007 - Getting small example AOI in location 'Berlin'.
2023-12-06 16:54:44,988 - Searching catalog with search_parameters: {'datetime': '2022-06-01T00:00:00Z/2022-12-31T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((13.375777664180191, 52.51799543512652), (13.375777664180191, 52.514561554285706), (13.381566455794598, 52.514561554285706), (13.381566455794598, 52.51799543512652), (13.375777664180191, 52.51799543512652)),)}, 'limit': 10, 'collections': ['phr'], 'query': {'cloudCoverage': {'lte': 20}}}
2023-12-06 16:54:45,532 - 5 results returned.


Output a dataframe with full scenes that match the specified parameters.

In [6]:
search_results_df
# catalog.search(as_dataframe=False) # Uncomment to output JSON instead

## Step 5. Fill out an order form

Fill out the order form for the chosen data product.

In [7]:
order_parameters = catalog.construct_order_parameters(
    data_product_id=data_product_id,
    image_id=search_results_df.iloc[0]["id"], # Note that the count starts from 0
    aoi=geometry,
)

2023-12-06 16:54:52,289 - See `catalog.get_data_product_schema(data_product_id)` for more detail on the parameter options.


Check the parameters by printing them.

In [8]:
order_parameters

{'dataProduct': '4f1b2f62-98df-4c74-81f4-5dce45deee99', 'params': {'id': 'a4c9e729-1b62-43be-82e4-4e02c31963dd', 'aoi': {'type': 'Polygon', 'coordinates': (((13.375966, 52.515068), (13.375966, 52.516639), (13.378314, 52.516639), (13.378314, 52.515068), (13.375966, 52.515068)),)}}}


## Step 6. Get a cost estimate

Get a cost estimation before placing a catalog order.

In [9]:
catalog.estimate_order(order_parameters)

2023-12-06 16:54:57,230 - Order is estimated to cost 150 UP42 credits (order_parameters: {'dataProduct': '4f1b2f62-98df-4c74-81f4-5dce45deee99', 'params': {'id': 'a4c9e729-1b62-43be-82e4-4e02c31963dd', 'aoi': {'type': 'Polygon', 'coordinates': (((13.375777664180191, 52.51799543512652), (13.375777664180191, 52.514561554285706), (13.381566455794598, 52.514561554285706), (13.381566455794598, 52.51799543512652), (13.375777664180191, 52.51799543512652)),)}}})


150

The response returns the overall credit amount that will be deducted from your credit balance if you decide to proceed with the ordering.

## Step 7. Place an order

Place an order with the requested parameters.

Credits will be deducted upon successful completion of the created catalog order. The transaction can't be reversed.

In [10]:
order = catalog.place_order(order_parameters)
order

2023-12-06 16:56:19,995 - Order is PLACED
2023-12-06 16:56:19,996 - Order 68567134-27ad-7bd7-4b65-d61adb11fc78 is now PLACED.


Order(order_id: 68567134-27ad-7bd7-4b65-d61adb11fc78, assets: [], dataProvider: None, status: PLACED, createdAt: 2023-12-06T15:56:17.268879Z, updatedAt: 2023-12-06T15:56:19.536492Z)

## Step 8. Monitor an order

Check the [status](https://docs.up42.com/developers/api-catalog/catalog-monitor#order-statuses) of your order. To learn about the timeframe of catalog orders, see [Asset delivery time](https://docs.up42.com/data/archive/delivery).

In [11]:
order.status

2023-12-06 16:57:32,203 - Order is PLACED


'PLACED'

You can also track the order status until the order is completed.

In [12]:
order.track_status()

The following statuses mean you can [download assets from storage](https://sdk.up42.com/notebooks/order-assets-example/):

- `BEING_FULFILLED`: some order assets might have been delivered.
- `FULFILLED`: all order assets have been delivered.